In [1]:
import tensorflow as tf
print("TensorFlow version: ", tf.__version__)
gpu = tf.config.experimental.list_physical_devices('GPU')
print("GPUs Available: ", len(gpu))
tf.config.experimental.set_memory_growth(gpu[0], True)
import pandas as pd
import xml.etree.ElementTree as ET
from PIL import Image
import os
import numpy as np
from keras import layers
from keras import models
from keras import utils
from keras_vggface.vggface import VGGFace
from keras.models import Model
from keras.callbacks import EarlyStopping
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import mtcnn
print(mtcnn.__version__)
import cv2
import sys
from IPython.display import clear_output
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import shutil
from IPython.display import display
from IPython.display import Image as Img
from tensorflow.keras import backend as K

TensorFlow version:  2.9.3
GPUs Available:  1
0.1.0


# 10.000 Imagens
## 5.000 conjuntos/Inputs  
Black(1000):               
500 Mulheres | 500 Homens: 250 iguais, 250 diferentes de cada   

White(1000):            
500 Mulheres | 500 Homens: 250 iguais, 250 diferentes de cada  

Indianos(1000):       
500 Mulheres | 500 Homens: 250 iguais, 250 diferentes de cada  

Asiaticos(1000):          
500 Mulheres | 500 Homens: 250 iguais, 250 diferentes de cada   

Outro(1000):                    
500 Mulheres | 500 Homens: 250 iguais, 250 diferentes de cada  


In [2]:
# vggface2 = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg', weights='vggface')

# inputs = vggface2.inputs
# outputs = vggface2.output
# model2 = Model(inputs, outputs)
# model2.compile()
# model2.summary()

In [4]:
#Definição da Siamese Convolutional Neural Network
vggface = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg', weights='vggface')
img_A = layers.Input((224, 224, 3), name = "Imagem Esquerda")
img_B = layers.Input((224, 224, 3), name = "Imagem Direita")
features_A = vggface(img_A)
features_B = vggface(img_B)
L1_layer = layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]), name = "L1_layer")
L1_distance = L1_layer([features_A, features_B])
model = Model(inputs = [img_A,img_B], outputs = L1_distance)
# for layer in model.layers[:-3]:
#     layer.trainable = False
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Imagem Esquerda (InputLayer)   [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Imagem Direita (InputLayer)    [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 vggface_resnet50 (Functional)  (None, 2048)         23561152    ['Imagem Esquerda[0][0]',        
                                                                  'Imagem Direita[0][0]']     

## Formato dos dados
### Como precisamos de duas imagens como input, teremos:
[X_train_A, X_train_B] e Y_train
### dessa forma:
[img_A, img_B] e AreTheyTheSame?
### Shape:
[(5000, 224, 224, 3), (5000, 224, 224, 3)] e (5000)

In [ ]:
x_train_a = []
x_train_b = []
y_train = []
amount = 504   #per ethnicity
# no treino pegaremos 504 imagens por etnia de modo que:
# 0-251 teremos comparacao entre diferentes imagens da mesma pessoa.            Ex: 0[0] com 0[1]   | 1[0] com 1[1] e assim por diante...
# 252-503 faremos comparacao entre diferentes imagens de pessoas diferentes.    Ex: 0[0] com 252[0] | 1[0] com 253[0] e assim por diante...
def trainBuild(x_train_a, x_train_b, y_train, amount, path):
    #print("Size of x_train = ",len(x_train[0]))
    all_folders = [f for f in os.listdir(path)]
    #print(all_folders)
    for folder in all_folders[:int(amount/2)]:
        counter = 0
        folder_path = os.path.join(path, folder)
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png'))]

        #pegaremos a primeira imagem da pasta da pessoa
        image_A = Image.open(folder_path+"/"+str(image_files[0]))
        image_A = image_A.resize((224,224))
        img_A_array = np.array(image_A)

        #pegaremos a segunda imagem da pasta da pessoa 
        image_B = Image.open(folder_path+"/"+str(image_files[1]))
        image_B = image_B.resize((224,224))
        img_B_array = np.array(image_B)

        #Duas imagens diferentes, mas da mesma pessoa
        x_train_a.append(img_A_array)
        x_train_b.append(img_B_array)
        #Como sao a mesma pessoa, sua label recebe 1
        y_train.append(1)
        counter += 1
    #as proximas 252 imagensA serao as primeiras 252 pessoas
    for folder in all_folders[:int(amount/2)]:
        counter = 0
        folder_path = os.path.join(path, folder)
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png'))]
        image_A = Image.open(folder_path+"/"+str(image_files[0]))
        image_A = image_A.resize((224,224))
        img_A_array = np.array(image_A)
        x_train_a.append(img_A_array)
        y_train.append(0)
        counter += 1
    #as proximas 252 imagensB serao as PROXIMAS 252 pessoas
    for folder in all_folders[int(amount/2):amount]:
        counter = 0
        folder_path = os.path.join(path, folder)
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png'))]
        image_B = Image.open(folder_path+"/"+str(image_files[0]))
        image_B = image_B.resize((224,224))
        img_B_array = np.array(image_B)
        x_train_b.append(img_B_array)
    return
trainBuild(x_train_a, x_train_b, y_train,amount, "F:/BiasBusters/Black_Male/Black_Male/")
trainBuild(x_train_a, x_train_b, y_train,amount, "F:/BiasBusters/Black_Female/Black_Female/")
trainBuild(x_train_a, x_train_b, y_train,amount, "F:/BiasBusters/Asian_Male/Asian_Male/")
trainBuild(x_train_a, x_train_b, y_train,amount, "F:/BiasBusters/Asian_Female/Asian_Female/")
trainBuild(x_train_a, x_train_b, y_train,amount, "F:/BiasBusters/White_Male/White_Male/")
trainBuild(x_train_a, x_train_b, y_train,amount, "F:/BiasBusters/White_Female/White_Female/")
trainBuild(x_train_a, x_train_b, y_train,amount, "F:/BiasBusters/Indian_Male/Indian_Male/")
trainBuild(x_train_a, x_train_b, y_train,amount, "F:/BiasBusters/Indian_Female/Indian_Female/")
trainBuild(x_train_a, x_train_b, y_train,amount, "F:/BiasBusters/Other_Male/Other_Male/")
trainBuild(x_train_a, x_train_b, y_train,amount, "F:/BiasBusters/Other_Female/Other_Female/")

print("Size of x_train_a = ",len(x_train_a))
print("Size of x_train_b = ",len(x_train_b))
print("Size of y_train   = ",len(y_train))

In [ ]:
X_train_A = np.array(x_train_a)
print(X_train_A.shape)
X_train_B = np.array(x_train_b)
print(X_train_B.shape)
# Y_train = np.array(y_train)
# print(y_train.shape)

In [ ]:
features_train = model.predict([X_train_A,X_train_B])
print("features_train.shape = "+ str(features_train.shape) + "y_train = " + str(y_train))
svm = SVC()
svm.fit(features_train, y_train)
print("Done Training")

In [ ]:
x_test_a = []
x_test_b = []
y_test = []
amount = 504   #per ethnicity
# no teste pegaremos 504 imagens por etnia de modo que:
# 504-755 teremos comparacao entre diferentes imagens da mesma pessoa.            Ex: 504[0] com 504[1]   | 505[0] com 505[1] e assim por diante...
# 756-1007 faremos comparacao entre diferentes imagens de pessoas diferentes.    Ex: 504[0] com 756[0] | 505[0] com 757[0] e assim por diante...
def testBuild(x_test_a, x_test_b, y_test, amount, path):
    #print("Size of x_test = ",len(x_test[0]))
    all_folders = [f for f in os.listdir(path)]
    #print(all_folders)
    #                                [504:756]
    for folder in all_folders[amount:amount+int(amount/2)]:
        counter = 0
        folder_path = os.path.join(path, folder)
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png'))]

        #pegaremos a primeira imagem da pasta da pessoa
        image_A = Image.open(folder_path+"/"+str(image_files[0]))
        image_A = image_A.resize((224,224))
        img_A_array = np.array(image_A)

        #pegaremos a segunda imagem da pasta da pessoa 
        image_B = Image.open(folder_path+"/"+str(image_files[1]))
        image_B = image_B.resize((224,224))
        img_B_array = np.array(image_B)

        #Duas imagens diferentes, mas da mesma pessoa
        x_test_a.append(img_A_array)
        x_test_b.append(img_B_array)
        #Como sao a mesma pessoa, sua label recebe 1
        y_test.append(1)
        counter += 1
    #as proximas 252 imagensA serao as primeiras 252 pessoas
    #                                [504:756]
    for folder in all_folders[amount:amount+int(amount/2)]:
        counter = 0
        folder_path = os.path.join(path, folder)
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png'))]
        image_A = Image.open(folder_path+"/"+str(image_files[0]))
        image_A = image_A.resize((224,224))
        img_A_array = np.array(image_A)
        x_test_a.append(img_A_array)
        y_test.append(0)
        counter += 1
    #as proximas 252 imagensB serao as PROXIMAS 252 pessoas
    #                                [756:1008]
    for folder in all_folders[amount+int(amount/2):]:
        counter = 0
        folder_path = os.path.join(path, folder)
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png'))]
        image_B = Image.open(folder_path+"/"+str(image_files[0]))
        image_B = image_B.resize((224,224))
        img_B_array = np.array(image_B)
        x_test_b.append(img_B_array)
    return
testBuild(x_test_a, x_test_b, y_test,amount, "F:/BiasBusters/Black_Male/Black_Male/")
testBuild(x_test_a, x_test_b, y_test,amount, "F:/BiasBusters/Black_Female/Black_Female/")
testBuild(x_test_a, x_test_b, y_test,amount, "F:/BiasBusters/Asian_Male/Asian_Male/")
testBuild(x_test_a, x_test_b, y_test,amount, "F:/BiasBusters/Asian_Female/Asian_Female/")
testBuild(x_test_a, x_test_b, y_test,amount, "F:/BiasBusters/White_Male/White_Male/")
testBuild(x_test_a, x_test_b, y_test,amount, "F:/BiasBusters/White_Female/White_Female/")
testBuild(x_test_a, x_test_b, y_test,amount, "F:/BiasBusters/Indian_Male/Indian_Male/")
testBuild(x_test_a, x_test_b, y_test,amount, "F:/BiasBusters/Indian_Female/Indian_Female/")
testBuild(x_test_a, x_test_b, y_test,amount, "F:/BiasBusters/Other_Male/Other_Male/")
testBuild(x_test_a, x_test_b, y_test,amount, "F:/BiasBusters/Other_Female/Other_Female/")

print("Size of x_test_a = ",len(x_test_a))
print("Size of x_test_b = ",len(x_test_b))
print("Size of y_test   = ",len(y_test))

In [ ]:
X_test_A = np.array(x_test_a)
print(X_test_A.shape)
X_test_B = np.array(x_test_b)
print(X_test_B.shape)
# Y_test = np.array(y_test)
# print(y_test.shape)

In [ ]:
features_test = model.predict([X_test_A,X_test_B])
accuracy = svm.score(features_test, y_test)
print(accuracy)